# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [129]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [117]:
weather_data = pd.read_csv("../starter_code/city_data.csv")

weather_df = pd.DataFrame(weather_data)
weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,0,Hongjiang,27.1100,109.9956,52.57,71,98,0.78
1,1,Mataura,-46.1927,168.8643,55.99,95,77,1.01
2,2,Longyearbyen,78.2186,15.6401,15.80,73,75,23.02
3,3,Ostrovnoy,68.0531,39.5131,11.89,93,89,10.27
4,4,Rikitea,-23.1203,-134.9692,80.01,75,6,12.17


In [118]:
gmaps.configure(api_key = g_key)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [128]:
locations = weather_df[['Latitude', 'Longitude']]

humidity = weather_df['Humidity'].astype(float)

In [127]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, 
                                    max_intensity=100, point_radius=3.0)

fig.add_layer(heatmap_layer)

plt.savefig("vacationheatmap.png")
fig

ValueError: weights must be of the same length as locations or None

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [121]:
ideal_weather_df = weather_data.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Temperature"] >= 70) & (weather_df["Temperature"] <= 80)].dropna()

ideal_weather_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
174,174,Tigre,-34.4260,-58.5796,70.00,68,0,4.61
257,257,Caravelas,-17.7125,-39.2481,76.12,82,0,5.50
275,275,Saldanha,-33.0117,17.9442,71.60,49,0,2.30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [122]:
ideal_weather_df['Hotel Name'] = ""

ideal_weather_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
174,174,Tigre,-34.4260,-58.5796,70.00,68,0,4.61,
257,257,Caravelas,-17.7125,-39.2481,76.12,82,0,5.50,
275,275,Saldanha,-33.0117,17.9442,71.60,49,0,2.30,


In [123]:
for index, row in ideal_weather_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }

        lat = row['Latitude']
        lng = row['Longitude']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        ideal_weather_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        ideal_weather_df.loc[index, "Hotel Name"] = "NaN"
        
ideal_weather_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
174,174,Tigre,-34.4260,-58.5796,70.00,68,0,4.61,Hotel Tigre
257,257,Caravelas,-17.7125,-39.2481,76.12,82,0,5.50,Mar Doce de Abrolhos
275,275,Saldanha,-33.0117,17.9442,71.60,49,0,2.30,12 on Beach Guest House


In [124]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
locations = ideal_weather_df[["Latitude", "Longitude"]]

In [115]:
# Add marker layer ontop of heat map
# Display figure

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))